[View in Colaboratory](https://colab.research.google.com/github/sierrajulietromeo/N12_plane_spotter/blob/master/N15_Plane_Tracker.ipynb)

# N12 Plane Tracker

I'm using a Raspberry Pi Zero W and a USB TV Tuner to pick up planes flying close to my house. Living in North London we are on the approach for Heathrow and I wanted to collect this data and do things with it.



In [1]:
!pip install gspread
!pip install gspread-dataframe
!pip install fiona

    100% |████████████████████████████████| 15.8MB 2.3MB/s 
    100% |████████████████████████████████| 71kB 22.9MB/s 
  Running setup.py bdist_wheel for munch ... - \ done
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [0]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe


In [0]:
from google.colab import auth
auth.authenticate_user()  # verifies your account to read files which you have access to. Make sure you have permission to read the file!
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default()) 

In [8]:
spreadsheet = gc.open("AC_data_2019") # Open file using its name. Use this if the file is already anywhere in your drive
sheet =  spreadsheet.get_worksheet(0)  # 0 means the first sheet in the file
df = pd.DataFrame(sheet.get_all_records())
df.head()



,First Altitude,First Latitude,First Longitude,First Seen,Flight,High Dist,High Rssi,Last Altitude,Last Latitude,Last Longitude,Last Seen,Low Dist,Low Rssi,Messages,Squawk,Transponder
0,0,4427774,0.040872,2018-09-23 19:58:37 Sunday,TAY029V,2662.2,-31.6,32025,44339539,0.040806,2018-09-23 20:05:05 Sunday,2658.4,-35.7,142,107,4cc0dc
1,40025,51545517,-0.649604,2018-09-23 20:21:15 Sunday,SVW39VV,3094.9,-31.9,39950,50795102,0.106659,2018-09-23 20:31:51 Sunday,3049.7,-35.5,205,6334,4d020f
2,39000,51805847,-0.420227,2018-09-23 20:43:38 Sunday,ETH503,3110.5,-31.9,38975,51634689,-0.005122,2018-09-23 20:51:39 Sunday,3100.1,-35.4,147,1160,04004c
3,39000,51764133,-0.405655,2018-09-23 21:09:19 Sunday,UAE232,3108,-31.9,39000,51631785,-0.006409,2018-09-23 21:15:41 Sunday,3099.9,-35.6,121,4742,89645b
4,38225,50551401,-0.542042,2018-09-23 19:58:37 Sunday,,3035.2,-32.2,31800,50551401,-0.542042,2018-09-23 20:01:24 Sunday,3035.2,-34.6,49,6303,3964f4


In [103]:
len(df)  # Checking number of records in the dataframe

514

In [104]:
print(list(df.columns))

['First Altitude', 'First Latitude', 'First Longitude', 'First Seen', 'Flight', 'High Dist', 'High Rssi', 'Last Altitude', 'Last Latitude', 'Last Longitude', 'Last Seen', 'Low Dist', 'Low Rssi', 'Messages', 'Squawk', 'Transponder']


In [105]:
# Create new dataframe based on DF of all the planes that I first detect between 5000 and 10000 ft high - 5000 might be a bit low
df2 = df[df['First Altitude'].between(5000, 10000, inclusive=True)]
#df2.head()
len(df2)

135

In [107]:
# Then refine that dataframe further to planes where last altitude detected was between 0-1000ft.   Now the dataset is of planes that overflew my house
df2 = df2[df2['Last Altitude'].between(0, 1000, inclusive=True)]
#df2.head(20)
len(df2)   # length of dataframe

39

In [11]:
import fiona
c = fiona.open('ne_10m_airports.shp', 'r')

OSError: ignored